Описание полей датасета hr_data.csv

1) Age – возраст сотрудников.

2) MonthlyIncome – ежемесячный доход сотрудников.

3) Department – отдел, в котором работает сотрудник.

4) Gender – пол сотрудника.

5) Education – уровень образования сотрудника.

6) Attrition (Текучесть кадров) – текучесть кадров, показывает, покинул ли сотрудник компанию (Yes – уволился, No – остался).



Задачи

1) Найти минимальный, максимальный и средний возраст сотрудников.

2) Подсчитать количество сотрудников в каждом отделе.

3) Рассчитать средний доход сотрудников в зависимости от уровня образования.

4) Рассчитать средний возраст сотрудников, которые уволились (Attrition = Yes) и которые остались (Attrition = No).

5) Вычислить средний доход по полу и сравнить разницу.

6) Найти отдел с наибольшим процентом уволенных сотрудников.

In [42]:
!pip3 install pyspark

In [43]:
# Вариант 2. Создаем дефолтную spark-сессию
from pyspark import SparkContext
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("RDD_HR").getOrCreate()

In [66]:
hr_source = spark.sparkContext.textFile("/content/hr_data.csv")

In [67]:
hr_source.take(5)

['Age,MonthlyIncome,Department,Gender,Education,Attrition',
 '41,5993,Sales,Female,2,Yes',
 '49,5130,Research & Development,Male,1,No',
 '37,2090,Research & Development,Male,2,Yes',
 '33,2909,Research & Development,Female,4,No']

In [68]:
# удаляем первую строчку
header = hr_source.first()
hr_source = hr_source.filter(lambda line: line != header)

In [69]:
hr_source.take(5)

['41,5993,Sales,Female,2,Yes',
 '49,5130,Research & Development,Male,1,No',
 '37,2090,Research & Development,Male,2,Yes',
 '33,2909,Research & Development,Female,4,No',
 '27,3468,Research & Development,Male,1,No']

In [70]:
# распаршиваем данные по столбцам
hr_source = hr_source.map(lambda line: line.split(","))

In [71]:
hr_source.take(5)

[['41', '5993', 'Sales', 'Female', '2', 'Yes'],
 ['49', '5130', 'Research & Development', 'Male', '1', 'No'],
 ['37', '2090', 'Research & Development', 'Male', '2', 'Yes'],
 ['33', '2909', 'Research & Development', 'Female', '4', 'No'],
 ['27', '3468', 'Research & Development', 'Male', '1', 'No']]

In [72]:
# преобразование типов данных

# Внимание, если вы делаете через dict преобразование значений, знайте, что в течение всей сессии пока данные не материализованы, они смотрят на этот словарь!
# Если вы его где-нибудь после кучи трансформаций переопределите, то все перестанет работать
map_dic = {
    'Female': 0,
    'Male': 1,
    'Yes': 1,
    'No': 0
}

hr = hr_source.map(lambda x: [float(x[0]), float(x[1]), x[2], map_dic[x[3]], int(x[4]), map_dic[x[5]]])

In [73]:
hr.take(5)

[[41.0, 5993.0, 'Sales', 0, 2, 1],
 [49.0, 5130.0, 'Research & Development', 1, 1, 0],
 [37.0, 2090.0, 'Research & Development', 1, 2, 1],
 [33.0, 2909.0, 'Research & Development', 0, 4, 0],
 [27.0, 3468.0, 'Research & Development', 1, 1, 0]]

Задание 1. Найти минимальный, максимальный и средний возраст сотрудников.

In [74]:
# Age - 1-ый столбец
hr.map(lambda x: x[0]).max(), hr.map(lambda x: x[0]).min(), hr.map(lambda x: x[0]).mean()

(60.0, 18.0, 36.923809523809545)

Задание 2. Подсчитать количество сотрудников в каждом отделе.

In [75]:
# Department - 3-ий столбец + нужен 1 любой столбец для расчетов
hr.map(lambda x: (x[2], x[0])).groupByKey().mapValues(lambda x: len(x)).take(5)

[('Sales', 446), ('Human Resources', 63), ('Research & Development', 961)]

Задание 3. Рассчитать средний доход сотрудников в зависимости от уровня образования.

In [76]:
# MonthlyIncome - 2-ой столбец + Education - 5-ый столбец
hr.map(lambda x: (x[4], x[1])).groupByKey().mapValues(lambda x: sum(x) / len(x)).take(5)

[(2, 6226.645390070922),
 (4, 6832.402010050251),
 (1, 5640.570588235294),
 (3, 6517.263986013986),
 (5, 8277.645833333334)]

Задание 4. Рассчитать средний возраст сотрудников, которые уволились (Attrition = Yes) и которые остались (Attrition = No).

In [80]:
# Attrition 6-ой столбец + Age 1-ый столбец
return_map_dic = {
    0: "No",
    1: "Yes"
}

hr.map(lambda x: (x[5], x[0])).groupByKey().mapValues(lambda x: sum(x) / len(x)).map(lambda x: (return_map_dic[x[0]], x[1])).take(5)

[('No', 37.561232765612324), ('Yes', 33.607594936708864)]

5. Вычислить средний доход по полу и сравнить разницу.

In [90]:
# Gender - 4-ый столбец + MonthlyIncome - 2-ой столбец
hr.map(lambda x: (x[3], x[1])).groupByKey().mapValues(lambda x: sum(x) / len(x)).map(lambda x: (1, x[1]))\
.groupByKey().mapValues(lambda x: list(x)[0] - list(x)[1]).map(lambda x: x[1]).collect()[0]

306.05839002267567

Задание 6. Найти отдел с наибольшим процентом уволенных сотрудников.

In [98]:
# Attrition 6-ой столбец + Department - 3-ий столбец

hr.map(lambda x: (x[2], x[5])).groupByKey().mapValues(lambda x: sum(x) / len(x)).map(lambda x: (x[0], round(x[1] * 100))).top(1, key=lambda x: x[1])

[('Sales', 21)]